<a href="https://colab.research.google.com/github/hBar2013/DS-Unit-1-Sprint-4-Statistical-Tests-and-Experiments/blob/master/module1-statistics-probability-and-inference/Kim_Lowry_LS_DS_141_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 3 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
### YOUR CODE STARTS HERE
import pandas as pd
from scipy.stats import ttest_ind, ttest_ind_from_stats, ttest_rel
import seaborn as sns
import numpy as np

In [0]:
congress_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data'
congress_df = pd.read_csv(congress_url)

In [64]:
congress_df.head()

,republican,n,y,n.1,y.1,y.2,y.3,n.2,n.3,n.4,y.4,?,y.5,y.6,y.7,n.5,y.8
0,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
1,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
2,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
3,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y
4,democrat,n,y,y,n,y,y,n,n,n,n,n,n,y,y,y,y


In [0]:
congress_df.rename(columns={congress_df.columns[0]: 'party',
                    congress_df.columns[1]: 'hc_infant',
                    congress_df.columns[2]: 'water_share',
                    congress_df.columns[3]: 'budget',
                    congress_df.columns[4]: 'doc_fee',
                    congress_df.columns[5]: 'el_salvadore',
                    congress_df.columns[6]: 'school_rel',
                    congress_df.columns[7]: 'antisat',
                    congress_df.columns[8]: 'contras',
                    congress_df.columns[9]: 'missles',
                    congress_df.columns[10]: 'immigration',
                    congress_df.columns[11]: 'synfuels',
                    congress_df.columns[12]: 'ed_spend',
                    congress_df.columns[13]: 'superfund_sue',
                    congress_df.columns[14]: 'crime',
                    congress_df.columns[15]: 'duty_free',
                    congress_df.columns[16]: 'sa_export'},
                    inplace=True)

In [66]:
congress_df.head()

,party,hc_infant,water_share,budget,doc_fee,el_salvadore,school_rel,antisat,contras,missles,immigration,synfuels,ed_spend,superfund_sue,crime,duty_free,sa_export
0,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
1,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
2,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
3,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y
4,democrat,n,y,y,n,y,y,n,n,n,n,n,n,y,y,y,y


In [0]:
#congress_df.set_index('party', inplace=True)

In [0]:
#congress_df.head()

In [0]:
congress_nans = congress_df.replace('?', np.nan)

In [70]:
congress_nans.head()

,party,hc_infant,water_share,budget,doc_fee,el_salvadore,school_rel,antisat,contras,missles,immigration,synfuels,ed_spend,superfund_sue,crime,duty_free,sa_export
0,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
1,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
2,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
3,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y
4,democrat,n,y,y,n,y,y,n,n,n,n,n,n,y,y,y,y


In [0]:
congress_num = congress_nans.replace({'y' : 1, 'n' : 0})

In [72]:
congress_num.head()

,party,hc_infant,water_share,budget,doc_fee,el_salvadore,school_rel,antisat,contras,missles,immigration,synfuels,ed_spend,superfund_sue,crime,duty_free,sa_export
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
1,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
2,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
3,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0
4,democrat,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0


In [0]:
gop = congress_num[congress_num['party'] == 'republican']

In [74]:
gop.describe()

,hc_infant,water_share,budget,doc_fee,el_salvadore,school_rel,antisat,contras,missles,immigration,synfuels,ed_spend,superfund_sue,crime,duty_free,sa_export
count,164.000000,147.000000,163.000000,164.000000,164.000000,165.000000,161.000000,156.000000,164.000000,164.000000,159.000000,154.000000,157.000000,160.000000,155.000000,145.000000
mean,0.189024,0.503401,0.134969,0.987805,0.951220,0.896970,0.242236,0.153846,0.115854,0.554878,0.132075,0.870130,0.859873,0.981250,0.090323,0.655172
std,0.392727,0.501698,0.342744,0.110092,0.216069,0.304924,0.429773,0.361963,0.321030,0.498501,0.339643,0.337257,0.348230,0.136067,0.287573,0.476960
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000
50%,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000
75%,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [0]:
dem = congress_num[congress_num['party'] == 'democrat']

In [76]:
dem.describe()

,hc_infant,water_share,budget,doc_fee,el_salvadore,school_rel,antisat,contras,missles,immigration,synfuels,ed_spend,superfund_sue,crime,duty_free,sa_export
count,258.000000,239.000000,260.000000,259.000000,255.000000,258.000000,259.000000,263.000000,248.000000,263.000000,255.000000,249.000000,252.000000,257.000000,251.000000,185.000000
mean,0.604651,0.502092,0.888462,0.054054,0.215686,0.476744,0.772201,0.828897,0.758065,0.471483,0.505882,0.144578,0.289683,0.350195,0.637450,0.935135
std,0.489876,0.501045,0.315405,0.226562,0.412106,0.500430,0.420224,0.377317,0.429121,0.500138,0.500949,0.352383,0.454518,0.477962,0.481697,0.246956
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [0]:
elsal_r = gop['el_salvadore']

In [0]:
elsal_d = dem['el_salvadore']

In [0]:
statistic, pvalue = ttest_ind(elsal_d, elsal_r, nan_policy='omit')

In [88]:
statistic

-21.06444879561732

In [89]:
pvalue

1.3035583587179729e-67

In [96]:
pvalue <= 0.5

True

In [0]:
infant_r = gop['hc_infant']

In [0]:
infant_d = dem['hc_infant']

In [0]:
statistic, pvalue = ttest_ind(infant_d, infant_r, nan_policy='omit')

In [94]:
statistic

9.15392122841775

In [95]:
pvalue

2.4195550274149624e-18

In [97]:
pvalue <= 0.5

True

In [0]:
water_r = gop['water_share']

In [0]:
water_d = dem['water_share']

In [0]:
statistic, pvalue = ttest_ind(water_d, water_r, nan_policy='omit')

In [101]:
statistic

-0.02491808700047811

In [102]:
pvalue

0.9801332440121653

In [103]:
pvalue <= 0.5

False